In [1]:
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
df = pd.read_csv('./../../data/processed/ID_Y.csv')
# df = pd.read_csv('./../../data/processed/.csv')

sales = df.loc[df.IMPORT == 1].groupby(['ID', 'YEAR'])['VART'].sum().unstack()
sales = sales.loc[sales.sum(1).sort_values().index]

logsales = np.log10(sales)
demlogsales = logsales.subtract(logsales.mean(1), axis = 0)

sizes = sales.sum(1)


Q = 10
parts = pd.cut(sizes.cumsum()/sizes.sum(), Q, labels = range(Q)).sort_index()

In [3]:
Sk = sales.groupby(parts).sum()
# yqs.T.cov().sum().sum()

X = Sk.sum() # == sales.sum()
EX = X.mean()

# Exact
print(Sk.T.cov().sum().sum())
print(X.var())

# Aggregate approx
print(np.log10(X).var()*(np.log(10)*EX)**2)

# Sectoral log fluctuations
# Sq = yqs.mean(1).mean() # approx un sq que es el valor lineal medio de los qs
# Sq**2*np.log(10)**2*(np.log10(yqs).T.cov().sum().sum())
print((EX/Q)**2*np.log(10)**2*(np.log10(Sk).T.cov().sum().sum()))

logsales = np.log10(sales)

micro_s = logsales.sub(logsales.mean(1), axis=0).unstack().std()
zero_shock = logsales.notna().multiply(logsales.mean(1), axis = 0).replace(0, np.nan)

# noise = np.power(10, zero_shock + simu_shocks) - np.power(10, zero_shock)
# common_R + noise_qs + base_qs - Sk == 0

nqs = parts.value_counts().values
lognqs = np.log10(nqs)

8.172793795363236e+21
8.172793795363238e+21
8.939063483056481e+21
9.696943094270655e+21


# Part 1:

In [ ]:
### The outcome of bootstrap is having the zero shock time series of quantiles and the 'all included time series of quantiles'.

n = 100
Q = 10


df = pd.read_csv('./../../data/processed/ID_Y.csv')
# df = pd.read_csv('./../../data/processed/.csv')


for i in [0, 1]:
    sales = df.loc[df.IMPORT == i].groupby(['ID', 'YEAR'])['VART'].sum().unstack()
    sales = sales.loc[sales.sum(1).sort_values().index]

#     sample = sales.sample(frac = .5); sample = sample.loc[sample.sum(1).sort_values().index] #sorting
#     sizes = sample.loc[sample.sum(1).sort_values().index].sum(1)
#     parts = pd.cut(sizes.cumsum()/sizes.sum(), Q, labels = range(Q)).sort_index()
#     eff_nq = sample.groupby(parts).count().mean(1).round().astype(int)

    base_out_list = []
    total_out_list = []


    for m in range(n):
        if m%10 == 0: print(m)

        df_bs_ = sales.sample(frac = .5)

        for size_sorting in [True, False]:

            for s in [.02, .05, .1, .25, .5]:

                if size_sorting: 
                    df_bs = df_bs_.loc[df_bs_.sum(1).sort_values().index] #sorting
                else:
                    df_bs = df_bs_

                logsales = np.log10(df_bs)

                micro_s = logsales.sub(logsales.mean(1), axis=0).unstack().std()
                zero_shock = logsales.notna().multiply(logsales.mean(1), axis = 0).replace(0, np.nan)

                simu_shocks = (s/micro_s)*logsales.sub(logsales.mean(1), axis = 0)

                total = np.power(10, zero_shock + simu_shocks)
                base = np.power(10, zero_shock)

                total['q'] = pd.cut(total.sum(1).cumsum(), Q, labels = range(Q))
                nqs = total['q'].value_counts().values



                base_yqs = base.groupby(total['q']).sum()
                total_yqs = total.groupby(total['q']).sum()        

                base_yqs['m'] = m; base_yqs['nqs'] = nqs; base_yqs['s'] = s; base_yqs['parts'] = ['P', 'Q'][size_sorting];
                base_out_list += [base_yqs]

                total_yqs['m'] = m; total_yqs['nqs'] = nqs; total_yqs['s'] = s; total_yqs['parts'] = ['P', 'Q'][size_sorting];
                total_out_list += [total_yqs]

    bs_base = pd.concat(base_out_list).reset_index().set_index(['q', 'm', 's', 'nqs'])
    bs_totl = pd.concat(total_out_list).reset_index().set_index(['q', 'm', 's', 'nqs'])

    # SAVE DATA
    bs_base.to_csv('./../../data/bootstraps/bs_base_m'+str(n)+'_q'+str(Q)+'_'+['X', 'M'][i]+'.csv')
    bs_totl.to_csv('./../../data/bootstraps/bs_totl_m'+str(n)+'_q'+str(Q)+'_'+['X', 'M'][i]+'.csv')


# Part 2:

In [ ]:
i = 1
#### LOAD DATA

n = 100
Q = 10

# legacy
# size_sorting = False
# bs_base = pd.read_csv('./../../data/bootstraps/bs_base_'+['random', 'sorted'][size_sorting]+'_m'+str(n)+'_q'+str(Q)+'csv', index_col=['q', 'm', 's', 'nqs'])
# bs_totl = pd.read_csv('./../../data/bootstraps/bs_totl_'+['random', 'sorted'][size_sorting]+'_m'+str(n)+'_q'+str(Q)+'csv', index_col=['q', 'm', 's', 'nqs'])

bs_base = pd.read_csv('./../../data/bootstraps/bs_base_m'+str(n)+'_q'+str(Q)+'_'+['X', 'M'][i]+'.csv', index_col=['q', 'm', 's', 'nqs'])
bs_totl = pd.read_csv('./../../data/bootstraps/bs_totl_m'+str(n)+'_q'+str(Q)+'_'+['X', 'M'][i]+'.csv', index_col=['q', 'm', 's', 'nqs'])

bs_base = bs_base.loc[bs_base.parts == 'P'].drop('parts', axis = 1)
bs_totl = bs_totl.loc[bs_totl.parts == 'P'].drop('parts', axis = 1)

### ADDITIVE DECOMPOSITION

bs_base = bs_base.groupby(level = ['s', 'q', 'm']).sum()  # drops the nq, it's not really grouping
bs_totl = bs_totl.groupby(level = ['s', 'q', 'm']).sum()  # drops the nq, it's not really grouping
fe_n = (bs_totl - bs_base)
fe = fe_n.groupby(level = ['s', 'q']).transform('median') 
noise = fe_n - fe

### MULTIPLICATIVE DECOMPOSITION

mul_fe_noise = bs_totl/bs_base
# Now, I use the bs to estimate disentangling fe from noise
mul_fe = mul_fe_noise.groupby(level = ['s', 'q']).transform('median') 
mul_noise = mul_fe_noise/mul_fe

# Important: they fulfill
# bs_totl - bs_base*mul_fe*mul_noise == 0
# np.log10(bs_totl) - np.log10(bs_base) - np.log10(mul_fe) - np.log10(mul_noise) == 0

In [ ]:
n = 100
Q = 10

# non-anonymous function
def pct10 (x): 
    return np.percentile(x, q=10)

def pct25 (x): 
    return np.percentile(x, q=25)

def pct75 (x): 
    return np.percentile(x, q=75)

def pct90 (x): 
    return np.percentile(x, q=90)

In [ ]:

#### LOAD DATA

for i in [0, 1]:
    for size_sorting in [True, False]:
    #     bs_base = pd.read_csv('./../../data/bootstraps/bs_base_'+['random', 'sorted'][size_sorting]+'_m'+str(n)+'_q'+str(Q)+'csv', index_col=['q', 'm', 's', 'nqs'])
    #     bs_totl = pd.read_csv('./../../data/bootstraps/bs_totl_'+['random', 'sorted'][size_sorting]+'_m'+str(n)+'_q'+str(Q)+'csv', index_col=['q', 'm', 's', 'nqs'])

        bs_base = pd.read_csv('./../../data/bootstraps/bs_base_m'+str(n)+'_q'+str(Q)+'_'+['X', 'M'][i]+'.csv', index_col=['q', 'm', 's', 'nqs'])
        bs_totl = pd.read_csv('./../../data/bootstraps/bs_totl_m'+str(n)+'_q'+str(Q)+'_'+['X', 'M'][i]+'.csv', index_col=['q', 'm', 's', 'nqs'])
        bs_base = bs_base.loc[bs_base.parts == ['P', 'Q'][size_sorting]].drop('parts', axis = 1)
        bs_totl = bs_totl.loc[bs_totl.parts == ['P', 'Q'][size_sorting]].drop('parts', axis = 1)

        ### ADDITIVE DECOMPOSITION

        bs_base = bs_base.groupby(level = ['s', 'q', 'm']).sum()  # drops the nq, it's not really grouping
        bs_totl = bs_totl.groupby(level = ['s', 'q', 'm']).sum()  # drops the nq, it's not really grouping
        fe_n = (bs_totl - bs_base)
        fe = fe_n.groupby(level = ['s', 'q']).transform('median') 
        noise = fe_n - fe

        ### MULTIPLICATIVE DECOMPOSITION

        mul_fe_noise = bs_totl/bs_base
        # Now, I use the bs to estimate disentangling fe from noise
        mul_fe = mul_fe_noise.groupby(level = ['s', 'q']).transform('median') 
        mul_noise = mul_fe_noise/mul_fe

        # Important: they fulfill
        # bs_totl - bs_base*mul_fe*mul_noise == 0
        # np.log10(bs_totl) - np.log10(bs_base) - np.log10(mul_fe) - np.log10(mul_noise) == 0

        for linear in [True, False]:

            if linear:
                BASE = bs_base
                FE = fe
                NOISE = noise
            else:
                BASE = np.log10(bs_base) # bs_base
                FE = np.log10(mul_fe) # fe
                NOISE = np.log10(mul_noise)# noise


            result_list = []
            obs_parts_cross_cov_list = []

            for m in range(n):
                if m%50 == 0: print(m)
                for s in [.02, .05, .1, .25, .5]:
                    base_m = BASE.loc[(BASE.index.get_level_values('m') == m) & (BASE.index.get_level_values('s') == s)]
                    fe_m = FE.loc[(FE.index.get_level_values('m') == m) & (FE.index.get_level_values('s') == s)]
                    n_m = NOISE.loc[(NOISE.index.get_level_values('m') == m) & (NOISE.index.get_level_values('s') == s)]

                    obs_s_m = pd.concat([base_m, fe_m, n_m])
                    obs_s_m.index = pd.MultiIndex.from_product([['base', 'FE', 'noise'],range(Q)], names = ['comp', 'part'])

                    cov = obs_s_m.T.cov() # shape: 3Q x 3Q

                    ####### OUTCOME 1
        #           Save observed cross covariance of the parts

    #                 obs_parts_cross_cov = cov.loc['noise'][['noise']].stack(); obs_parts_cross_cov.index.names = ['partA', 'partB']; obs_parts_cross_cov.columns = ['cov']
    #                 obs_parts_cross_cov['m'] = m; obs_parts_cross_cov['s'] = s
    #                 obs_parts_cross_cov = obs_parts_cross_cov.reset_index().set_index(['s', 'm', 'partA', 'partB'])

    #                 obs_parts_cross_cov_list += [obs_parts_cross_cov]

                    obs_parts_cross_cov = pd.DataFrame(cov.stack([0, 1]), columns = ['cov'])
                    obs_parts_cross_cov.index.names = ['compA', 'partA', 'compB', 'partB']
                    obs_parts_cross_cov['m'] = m; obs_parts_cross_cov['s'] = s
                    obs_parts_cross_cov = obs_parts_cross_cov.reset_index().set_index(['s', 'm', 'compA', 'partA', 'compB', 'partB'])

                    obs_parts_cross_cov_list += [obs_parts_cross_cov]


                    # np.log10(abs(obs_s_m.T.cov())).round(1)

                    ####### OUTCOME 2
                    # Characterize diagonal and offdiagonal for all 3x3 cross components

                    ## Make a mask for i = j
                    ## Groupby the other level and (we need to sum) but possibly describe.
                    part_index = np.array(3*Q*[cov.index.get_level_values('part').values])

                    ## Mask for off diagonals
                    part_diag_mask = np.equal(part_index, part_index.T)
                    pd_diag_mask = pd.DataFrame(part_diag_mask, index = cov.index, columns = cov.columns)


                    diag = cov[pd_diag_mask].stack([-2, -1]).groupby(level = [0, 2]).agg(['count', np.mean, np.sum, np.std, 
                                                                                          pct10, pct25, pct75, pct90])
                    diag = pd.concat([diag], keys=['diag'], names=['diag'], axis = 0)

                    offdiag = cov[~pd_diag_mask].stack([-2, -1]).groupby(level = [0, 2]).agg(['count', np.mean, np.sum, np.std,
                                                                                          pct10, pct25, pct75, pct90])
                    offdiag = pd.concat([offdiag], keys=['offdiag'], names=['diag'], axis = 0)


                    result_m = pd.concat([diag, offdiag])
                    result_m = pd.concat([result_m], keys=[m], names=['m'], axis = 0)
                    result_m = pd.concat([result_m], keys=[s], names=['s'], axis = 0)
                    result_m.index.names = ['s', 'm', 'diag', 'compA', 'compB']

                    result_list += [result_m]

            result_varname = '_'.join(['cov_elements_desc', ['random', 'sorted'][size_sorting], ['log', 'lin'][linear], ['X', 'M'][i]]); print(result_varname)
            result = pd.concat(result_list)
            result.to_csv('./../../data/bootstraps/'+result_varname+'.csv')
            globals()[result_varname] = result


            result_varname = '_'.join(['parts_cross_cov', ['random', 'sorted'][size_sorting], ['log', 'lin'][linear], ['X', 'M'][i]]); print(result_varname)
            result = pd.concat(obs_parts_cross_cov_list)
            result.to_csv('./../../data/bootstraps/'+result_varname+'.csv')            
            globals()[result_varname] = result

    #         if linear:
    #             result_lin = pd.concat(result_list)
    #             result_parts_cross_cov_lin = pd.concat(obs_parts_cross_cov_list) ## adapt
    #         else:
    #             result_log = pd.concat(result_list)
    #             result_parts_cross_cov_log = pd.concat(obs_parts_cross_cov_list)


In [ ]:
# NameError: name 'result_log' is not defined
result_log_scaled = result_log.copy()
result_log_scaled[['mean','sum','std']] = component_scale_factor*result_log[['mean','sum','std']]

component_scale_factor = (((Sk/2).mean(1)*np.log(10))**2).mean()   ## This is (Sq*ln10)**2, but Sq/2 because of BS
component_scale_factor